# Assignment5: CRM Cleanup @ **DalaShop**

_Files (Ch.10), Exceptions (Ch.10), Unit Tests (Ch.11), and Regular Expressions_  
.....

**Total: 3 points** (Two questions, 1.5 pts each)

> This assignment is scenario-based and aligned with Python Crash Course Ch.10 (files & exceptions), Ch.11 (unit testing with `unittest`), and Regular Expressions.


## Scenario

You are a data intern at an online retailer called **DalaShop**.  
Sales exported a **raw contacts** file from the CRM. It contains customer names, emails, and phone numbers, but the formatting is messy and some emails are invalid.  
Your tasks:

1. **Clean** the contacts (Files + Exceptions + Regex).
2. **Write unit tests** to make sure your helper functions work correctly and keep working in the future.


## Data file (given by the company): `contacts_raw.txt`

Use this exact sample data (you may extend it for your own testing, but do **not** change it when submitting).  
Run the next cell once to create the file beside your notebook.


In [ ]:
# Create the provided company dataset file
with open("contacts_raw.txt", "w", encoding="utf-8") as f:
    f.write('Alice Johnson <alice@example.com> , +1 (469) 555-1234\nBob Roberts <bob[at]example.com> , 972-555-777\nSara M. , sara@mail.co , 214 555 8888\n"Mehdi A." <mehdi.ay@example.org> , (469)555-9999\nDelaram <delaram@example.io>, +1-972-777-2121\nNima <NIMA@example.io> , 972.777.2121\nduplicate <Alice@Example.com> , 469 555 1234')
print("Wrote contacts_raw.txt with sample DalaShop data.")

## Q1 (1.5 pts) — CRM cleanup with Files, Exceptions, and Regex

Implement `q1_crm_cleanup.py` to:

1. **Read** `contacts_raw.txt` using `pathlib` and `with`. If the file is missing, **handle** it gracefully with `try/except FileNotFoundError` (print a friendly message; do not crash).
2. **Validate emails** with a simple regex (`r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}"`).
   - Trim whitespace with `strip()` before checking.
   - Use **full** matching (not partial).
3. **Normalize phone numbers:** remove all non-digits (e.g., with `re.sub(r"\D", "", raw)`).
   - If the result has **≥ 10 digits**, keep the **last 10 digits**.
   - Otherwise, return an **empty string** (`""`).
4. **Filter rows:** keep **only** rows with a valid email.
5. **Deduplicate:** remove duplicates by **email** using **case-insensitive** comparison (e.g., `email.casefold()`). **Keep the first occurrence** and drop later duplicates.
6. **Output CSV:** write to `contacts_clean.csv` with **columns exactly** `name,email,phone` (UTF-8).
7. **Preserve input order:** the order of rows in `contacts_clean.csv` must match the **first appearance** order from the input file. **Do not sort** the rows.

**Grading rubric (1.5 pts):**

- (0.4) File read/write via `pathlib` + graceful `FileNotFoundError` handling
- (0.5) Correct email regex validation + filtering
- (0.4) Phone normalization + case-insensitive de-dup (keep first)
- (0.2) Clean code, clear names, minimal docstrings/comments


In [1]:
# Write your answer here
import unittest
from pathlib import Path
import csv
import q1_crm_cleanup as crm


class TestCrmCleanupFileFlow(unittest.TestCase):
    def setUp(self):
        self.tmp_dir = Path(".").resolve()
        self.input_path = self.tmp_dir / "contacts_raw.txt"
        self.output_path = self.tmp_dir / "contacts_clean.csv"

        sample = (
            "Alice, alice@example.com, (214) 555-1212\n"
            "Bob, bob@example.com, 972-777-8888\n"
            "Bad, bad-email, 111-222-3333\n"
            "Dup, ALICE@EXAMPLE.COM, 999-999-9999\n"
            "NoPhone, nop@school.com,\n"
        )
        self.input_path.write_text(sample, encoding="utf-8")

    def tearDown(self):
        # clean up files if they exist
        if self.input_path.exists():
            self.input_path.unlink()
        if self.output_path.exists():
            self.output_path.unlink()

    def test_parse_contacts_from_file_ok(self):
        rows = crm.parse_contacts_from_file(str(self.input_path))
        
        expected = [
            {"name": "Alice", "email": "alice@example.com", "phone": "2145551212"},
            {"name": "Bob", "email": "bob@example.com", "phone": "9727778888"},
            {"name": "NoPhone", "email": "nop@school.com", "phone": ""},
        ]
        self.assertEqual(rows, expected)

    def test_parse_contacts_from_file_missing(self):
        missing_path = "does_not_exist.txt"
        rows = crm.parse_contacts_from_file(missing_path)
        self.assertEqual(rows, [])  # should not crash and return empty

    def test_write_contacts_csv(self):
        rows = [
            {"name": "Alice", "email": "alice@example.com", "phone": "2145551212"},
            {"name": "Bob", "email": "bob@example.com", "phone": "9727778888"},
            {"name": "NoPhone", "email": "nop@school.com", "phone": ""},
        ]
        crm.write_contacts_csv(rows, str(self.output_path))

        # check file 
        self.assertTrue(self.output_path.exists())

        
        with self.output_path.open("r", encoding="utf-8", newline="") as f:
            reader = csv.reader(f)
            rows_read = list(reader)

        self.assertEqual(rows_read[0], ["name", "email", "phone"])
        # remaining rows must match in order
        self.assertEqual(
            rows_read[1:],
            [
                ["Alice", "alice@example.com", "2145551212"],
                ["Bob", "bob@example.com", "9727778888"],
                ["NoPhone", "nop@school.com", ""],
            ],
        )

if __name__ == "__main__":
    unittest.main(argv=["first-arg-is-ignored"], exit=False)




..

.
----------------------------------------------------------------------
Ran 3 tests in 0.016s

OK


File not found: does_not_exist.txt


## Q2 (1.5 pts) — Unit testing with `unittest`

Create tests in `test_crm_cleanup.py` that cover at least:

1. **Email validation**: valid/invalid variations.
2. **Phone normalization**: parentheses, dashes, spaces, country code; too-short cases.
3. **Parsing**: from a small multi-line string (not from a file), assert the exact structured rows (name/email/phone).
4. **De-duplication**: demonstrate that a case-variant duplicate email is dropped (first occurrence kept).


In [1]:
# Write your answer here
import unittest
import q2_crm_cleanup as crm



class TestCrmCleanup(unittest.TestCase):
    def test_email_validation(self):
        self.assertEqual(crm.validate_email("test@example.com"), "test@example.com")
        self.assertEqual(crm.validate_email(" user+tag@domain.co.uk "), "user+tag@domain.co.uk")
        self.assertIsNone(crm.validate_email("bad-email"))
        self.assertIsNone(crm.validate_email("user@domain"))
        self.assertIsNone(crm.validate_email("user@domain.c"))
        self.assertIsNone(crm.validate_email("user@domain.com extra"))

    def test_phone_normalization(self):
        self.assertEqual(crm.normalize_phone("(214) 555-1212"), "2145551212")
        self.assertEqual(crm.normalize_phone("214-555-1212"), "2145551212")
        self.assertEqual(crm.normalize_phone("214 555 1212"), "2145551212")
        self.assertEqual(crm.normalize_phone("+1 (214) 555-1212"), "2145551212")
        self.assertEqual(crm.normalize_phone("001-214-555-1212"), "2145551212")
        self.assertEqual(crm.normalize_phone("123456789"), "")
        self.assertEqual(crm.normalize_phone(""), "")

    def test_parsing_from_string(self):
        text = (
            "Alice, alice@example.com, (214) 555-1212\n"
            "Bob, bob@example.com, 972-777-8888\n"
            "Bad, bad-email, 111-222-3333\n"
        )
        rows = crm.parse_contacts_from_string(text)
        expected = [
            {"name": "Alice", "email": "alice@example.com", "phone": "2145551212"},
            {"name": "Bob", "email": "bob@example.com", "phone": "9727778888"},
        ]
        self.assertEqual(rows, expected)

    def test_deduplication_case_insensitive(self):
        rows = [
            {"name": "Sam", "email": "sam@example.com", "phone": "2145551212"},
            {"name": "SAM DUP", "email": "SAM@example.com", "phone": "9727778888"},
            {"name": "Other", "email": "other@example.com", "phone": "4690001111"},
        ]
        deduped = crm.deduplicate_contacts(rows)
        self.assertEqual(len(deduped), 2)
        emails = [r["email"] for r in deduped]
        self.assertIn("sam@example.com", emails)
        self.assertIn("other@example.com", emails)
        self.assertNotIn("SAM@example.com", emails)


if __name__ == "__main__":
    unittest.main(argv=["first-arg-is-ignored"], exit=False)





....
----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


## Grading rubric (total 3 pts)

- **Q1 (1.5 pts)**
  - (0.4) File I/O with `pathlib` + graceful `FileNotFoundError` handling
  - (0.5) Email validation (regex + strip + full match) and filtering
  - (0.4) Phone normalization and **case-insensitive** de-duplication (keep first)
  - (0.2) Code clarity (names, minimal docstrings/comments)
- **Q2 (1.5 pts)**
  - (0.6) Meaningful coverage for email/phone functions (valid & invalid)
  - (0.6) Parsing & de-dup tests that assert exact expected rows
  - (0.3) Standard `unittest` structure and readable test names
